In [1]:
import pandas as pd
import glob
import datetime
import re
import warnings
import numpy as np
warnings.filterwarnings(action='ignore')

## 버스 데이터 파일 불러오기

In [2]:
file_list = glob.glob('./data/일반*.xlsx')
# file_list

In [3]:
# 정류장 번호
place = {
    '경주골굴사':['02089','02090'], '경주김유신묘':['01025','01036'], 
    '경주동궁과월지':['01237', '01238'], '경주동부사적지대':['01279','01280'], 
    '경주무열왕릉':['01194','01195'], '경주문무대왕릉':['02056','02057'],
    '경주불국사':['01160','01161'], '경주석굴암':['01199','01199'], 
    '경주신문왕릉':['01081','01082'], '경주월성(반월성)':['01237','01238'], 
    '경주첨성대':['01279','01280'], '경주금곡사':['05379','05380'],
    '경주기림사':['02021','02021'], '경주백률사':['01228','01229'],
    '경주분황사':['01157','01158'],'경주연지암':['05423','05426'], 
    '경주주사암':['03121','03122']
} 

In [4]:
df = pd.read_excel(file_list[0])
del df['Unnamed: 0']

## '도착시간' 시간 계속 더해가기

In [5]:
time = int(df['도착시간'][1].split('분')[0])
sum_time = 0
for i in range(1, len(df['도착시간'])):
    time = int(df['도착시간'][i].split('분')[0])
    sum_time += time
    df['도착시간'].loc[i] = sum_time
df

,버스노선,정류장번호,버스번호,도착시간,배차간격
0,시외버스.고속버스터미널,1225,일반10,06:00,평일 20분 8회
1,고속버스.시외버스터미널,1045,일반10,1,평일 20분 8회
2,서라벌사거리.황리단길,1190,일반10,4,평일 20분 8회
3,중앙시장,1309,일반10,5,평일 20분 8회
4,법원.동산병원 건너,1083,일반10,6,평일 20분 8회
5,신한은행사거리,1232,일반10,8,평일 20분 8회
6,경주우체국,1031,일반10,9,평일 20분 8회
7,경고지하도,1007,일반10,10,평일 20분 8회
8,팔우정,1351,일반10,11,평일 20분 8회
9,월성동주민센타,1280,일반10,12,평일 20분 8회


## 시간 변환 함수

In [6]:
def time_convert(time):
    hour = 0
    minute = 0
    second = 0
    if time >= 60:
        hour = time // 60
        minute = time % 60
    else:
        minute = time
    return [hour, minute]

## 배차 간격에 따른 컬럼 추가

In [7]:
# 운행 횟수
baecha_count = int(re.sub(r'[^0-9]','', df['배차간격'][0].split('분')[1])) - 1

# 배차간격
baecha_time = int(re.sub(r'[^0-9]','', df['배차간격'][0].split('분')[0]))
print(df['배차간격'][0], ' -> ', baecha_time, baecha_count)

평일 20분 8회  ->  20 7


In [8]:
# 배차간격 시간 변환 함수에 넣어서 시간 계산
baecha_time_sum = []
tmp = 0
for i in range(baecha_count):
    tmp += baecha_time
    baecha_time_sum.append(tmp)
baecha_time_sum

[20, 40, 60, 80, 100, 120, 140]

In [9]:
for i in range(baecha_count):
    df[f'배차{i}'] = df['도착시간']
    time_test = time_convert(baecha_time_sum[i])
    df[f'배차{i}'][0] = str(int(df[f'배차{i}'][0].split(':')[0]) + time_test[0]) + ':' + str(int(df[f'배차{i}'][0].split(':')[1]) + time_test[1])
   
    for n in range(1, len(df)):
        df[f'배차{i}'][n] = df['도착시간'][n]
        a = time_convert(df['도착시간'][n])
        df[f'배차{i}'][n] = str(int(df[f'배차{i}'][0].split(':')[0]) + a[0]) + ':' + str(int(df[f'배차{i}'][0].split(':')[1]) + a[1])
        if int(df[f'배차{i}'][n].split(':')[1]) >= 60:
            b = time_convert(int(df[f'배차{i}'][n].split(':')[1]))
            df[f'배차{i}'][n] = str(int(df[f'배차{i}'][0].split(':')[0]) + b[0]) + ':' + str(b[1])
        
df

,버스노선,정류장번호,버스번호,도착시간,배차간격,배차0,배차1,배차2,배차3,배차4,배차5,배차6
0,시외버스.고속버스터미널,1225,일반10,06:00,평일 20분 8회,6:20,6:40,7:0,7:20,7:40,8:0,8:20
1,고속버스.시외버스터미널,1045,일반10,1,평일 20분 8회,6:21,6:41,7:1,7:21,7:41,8:1,8:21
2,서라벌사거리.황리단길,1190,일반10,4,평일 20분 8회,6:24,6:44,7:4,7:24,7:44,8:4,8:24
3,중앙시장,1309,일반10,5,평일 20분 8회,6:25,6:45,7:5,7:25,7:45,8:5,8:25
4,법원.동산병원 건너,1083,일반10,6,평일 20분 8회,6:26,6:46,7:6,7:26,7:46,8:6,8:26
5,신한은행사거리,1232,일반10,8,평일 20분 8회,6:28,6:48,7:8,7:28,7:48,8:8,8:28
6,경주우체국,1031,일반10,9,평일 20분 8회,6:29,6:49,7:9,7:29,7:49,8:9,8:29
7,경고지하도,1007,일반10,10,평일 20분 8회,6:30,6:50,7:10,7:30,7:50,8:10,8:30
8,팔우정,1351,일반10,11,평일 20분 8회,6:31,6:51,7:11,7:31,7:51,8:11,8:31
9,월성동주민센타,1280,일반10,12,평일 20분 8회,6:32,6:52,7:12,7:32,7:52,8:12,8:32


## 장소와 시간 입력에 따른 데이터 접근 

In [22]:
# 장소 입력
print(list(place.keys()))
input_place = input('장소 입력: ')

['경주골굴사', '경주김유신묘', '경주동궁과월지', '경주동부사적지대', '경주무열왕릉', '경주문무대왕릉', '경주불국사', '경주석굴암', '경주신문왕릉', '경주월성(반월성)', '경주첨성대', '경주금곡사', '경주기림사', '경주백률사', '경주분황사', '경주연지암', '경주주사암']
장소 입력: 경주분황사


In [23]:
place[input_place]  # 유적지와 가장 가까운 정류장 번호

['01157', '01158']

In [24]:
# 시간 입력
input_hour, input_minute =  map(int, input().split())

6 30


## 유적지 지나는 버스 가져오기

In [25]:
# 데이터프레임에 '정류장번호' string 변환 후 리스트에 저장
str_num = []
for x in df['정류장번호']:
    str_num.append(str(x).zfill(5))
str_num[:4]

['01225', '01045', '01190', '01309']

In [26]:
# 몇 번 버스 타야하는지 출력
time_data = []
name_data = []

# 입력한 유적지를 지나가는 버스 데이터 가져오기
for x in range(len(str_num)):
    for y in range(2):
        if str_num[x] == place[input_place][y]:
            print(df['버스번호'][0], df['버스노선'][x])
            name_data.append(df['버스노선'][x])

    # 입력한 시간 이후의 버스 도착시간 가져오기
            for z in range(baecha_count):
                hour = int(df[f'배차{z}'][x].split(':')[0])
                minute = int(df[f'배차{z}'][x].split(':')[1])
                if hour >= input_hour and minute >= input_minute:
                    time_data.append(df.loc[x][f'배차{z}'])

result = {'버스': df['버스번호'][0],
          '정류장':name_data,
          '시간표':time_data}
print(result)
    

일반10 분황사
{'버스': '일반10', '정류장': ['분황사'], '시간표': ['6:37', '6:57', '7:37', '7:57', '8:37']}


## 한 번에 가는 코스
* n개의 코스 입력
* 두 유적지가 있는 버스 데이터를 가져옴
 - 조건 1. 먼저 입력했던 유적지가 앞에 있어야 함
* 조건 1을 반복하면서 n-1개의 버스를 구함

## 환승 함수 구하기

* 모든 버스 데이터에 접근해서 중복되는 정류장 번호를 찾음
 - 조건 1. 처음 버스는 중복 정류장보다 앞에 있어야 함
 - 조건 2. 다음 버스는 중복 정류장보다 뒤에 있어야 함

## 모든 버스 데이터 가져와서 for문으로 돌리는 코드